In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from sqlalchemy import create_engine
import psycopg2
from config import password
import numpy as np

In [2]:
avgattendance ='''<table cellpadding="3" cellspacing="1" class="tablehead">
					<tbody><tr class="stathead"><td colspan="2">2020 Team</td><td colspan="4">Home</td><td colspan="3">Road</td><td colspan="3">Overall</td></tr><tr class="colhead"><td align="left" title="Rank">RK</td><td align="left" title="">TEAM</td><td align="right" title="Games">GMS</td><td align="right" title=""><a href="//www.espn.com/nba/attendance/_/sort/homeTotal">TOTAL</a></td><td align="right" title="Average"><a href="//www.espn.com/nba/attendance/_/order/false">AVG</a></td><td align="right" title="Percentage"><a href="//www.espn.com/nba/attendance/_/sort/homePct">PCT</a></td><td align="right" title="Games">GMS </td><td align="right" title="Average"><a href="//www.espn.com/nba/attendance/_/sort/awayAvg">AVG </a></td><td align="right" title="Percentage"><a href="//www.espn.com/nba/attendance/_/sort/awayPct">PCT </a></td><td align="right" title="Games"> GMS</td><td align="right" title="Average"><a href="//www.espn.com/nba/attendance/_/sort/allAvg"> AVG</a></td><td align="right" title="Percentage"><a href="//www.espn.com/nba/attendance/_/sort/allPct"> PCT</a></td></tr><tr class="oddrow team-46-20"><td>1</td><td><a href="http://www.espn.com/nba/team/_/name/phi/philadelphia-76ers">76ers</a></td><td align="right">31</td><td align="right">639,491</td><td align="right" class="sortcell">20,628</td><td align="right">102.4</td><td align="right">34</td><td align="right">17,706</td><td align="right">94.5</td><td align="right">65</td><td align="right">19,122</td><td align="right">98.5</td></tr><tr class="evenrow team-46-6"><td>2</td><td><a href="http://www.espn.com/nba/team/_/name/dal/dallas-mavericks">Mavericks</a></td><td align="right">34</td><td align="right">682,096</td><td align="right" class="sortcell">20,061</td><td align="right">104.5</td><td align="right">33</td><td align="right">18,249</td><td align="right">97.8</td><td align="right">67</td><td align="right">19,169</td><td align="right">101.2</td></tr><tr class="oddrow team-46-28"><td>3</td><td><a href="http://www.espn.com/nba/team/_/name/tor/toronto-raptors">Raptors</a></td><td align="right">32</td><td align="right">633,456</td><td align="right" class="sortcell">19,795</td><td align="right">100.0</td><td align="right">32</td><td align="right">17,789</td><td align="right">94.9</td><td align="right">64</td><td align="right">18,792</td><td align="right">97.5</td></tr><tr class="evenrow team-46-14"><td>4</td><td><a href="http://www.espn.com/nba/team/_/name/mia/miami-heat">Heat</a></td><td align="right">32</td><td align="right">629,771</td><td align="right" class="sortcell">19,680</td><td align="right">100.4</td><td align="right">33</td><td align="right">18,127</td><td align="right">96.8</td><td align="right">65</td><td align="right">18,891</td><td align="right">98.6</td></tr><tr class="oddrow team-46-22"><td>5</td><td><a href="http://www.espn.com/nba/team/_/name/por/portland-trail-blazers">Trail Blazers</a></td><td align="right">32</td><td align="right">608,634</td><td align="right" class="sortcell">19,633</td><td align="right">101.0</td><td align="right">34</td><td align="right">17,941</td><td align="right">96.6</td><td align="right">66</td><td align="right">18,748</td><td align="right">98.8</td></tr><tr class="evenrow team-46-7"><td>6</td><td><a href="http://www.espn.com/nba/team/_/name/den/denver-nuggets">Nuggets</a></td><td align="right">33</td><td align="right">633,153</td><td align="right" class="sortcell">19,186</td><td align="right">98.3</td><td align="right">32</td><td align="right">17,163</td><td align="right">92.8</td><td align="right">65</td><td align="right">18,190</td><td align="right">95.7</td></tr><tr class="oddrow team-46-2"><td>7</td><td><a href="http://www.espn.com/nba/team/_/name/bos/boston-celtics">Celtics</a></td><td align="right">32</td><td align="right">610,864</td><td align="right" class="sortcell">19,089</td><td align="right">102.5</td><td align="right">32</td><td align="right">18,409</td><td align="right">97.7</td><td align="right">64</td><td align="right">18,749</td><td align="right">100.1</td></tr><tr class="evenrow team-46-12"><td>8</td><td><a href="http://www.espn.com/nba/team/_/name/lac/la-clippers">Clippers</a></td><td align="right">32</td><td align="right">610,176</td><td align="right" class="sortcell">19,068</td><td align="right">100.0</td><td align="right">32</td><td align="right">17,855</td><td align="right">96.2</td><td align="right">64</td><td align="right">18,461</td><td align="right">98.1</td></tr><tr class="oddrow team-46-13"><td>9</td><td><a href="http://www.espn.com/nba/team/_/name/lal/los-angeles-lakers">Lakers</a></td><td align="right">31</td><td align="right">588,907</td><td align="right" class="sortcell">18,997</td><td align="right">99.6</td><td align="right">32</td><td align="right">18,877</td><td align="right">101.9</td><td align="right">63</td><td align="right">18,936</td><td align="right">100.8</td></tr><tr class="evenrow team-46-18"><td>10</td><td><a href="http://www.espn.com/nba/team/_/name/ny/new-york-knicks">NY Knicks</a></td><td align="right">33</td><td align="right">620,789</td><td align="right" class="sortcell">18,811</td><td align="right">95.0</td><td align="right">33</td><td align="right">17,767</td><td align="right">94.1</td><td align="right">66</td><td align="right">18,289</td><td align="right">94.5</td></tr><tr class="oddrow team-46-4"><td>11</td><td><a href="http://www.espn.com/nba/team/_/name/chi/chicago-bulls">Bulls</a></td><td align="right">34</td><td align="right">639,352</td><td align="right" class="sortcell">18,804</td><td align="right">89.9</td><td align="right">31</td><td align="right">17,767</td><td align="right">94.3</td><td align="right">65</td><td align="right">18,310</td><td align="right">91.9</td></tr><tr class="evenrow team-46-24"><td>12</td><td><a href="http://www.espn.com/nba/team/_/name/sa/san-antonio-spurs">Spurs</a></td><td align="right">30</td><td align="right">550,515</td><td align="right" class="sortcell">18,350</td><td align="right">100.0</td><td align="right">33</td><td align="right">17,588</td><td align="right">93.7</td><td align="right">63</td><td align="right">17,951</td><td align="right">96.7</td></tr><tr class="oddrow team-46-26"><td>13</td><td><a href="http://www.espn.com/nba/team/_/name/utah/utah-jazz">Jazz</a></td><td align="right">31</td><td align="right">567,486</td><td align="right" class="sortcell">18,306</td><td align="right">100.0</td><td align="right">33</td><td align="right">17,261</td><td align="right">92.2</td><td align="right">64</td><td align="right">17,767</td><td align="right">95.9</td></tr><tr class="evenrow team-46-25"><td>14</td><td><a href="http://www.espn.com/nba/team/_/name/okc/oklahoma-city-thunder">Thunder</a></td><td align="right">33</td><td align="right">600,699</td><td align="right" class="sortcell">18,203</td><td align="right">100.0</td><td align="right">31</td><td align="right">17,855</td><td align="right">95.6</td><td align="right">64</td><td align="right">18,034</td><td align="right">97.8</td></tr><tr class="oddrow team-46-10"><td>15</td><td><a href="http://www.espn.com/nba/team/_/name/hou/houston-rockets">Rockets</a></td><td align="right">32</td><td align="right">578,458</td><td align="right" class="sortcell">18,076</td><td align="right">100.1</td><td align="right">32</td><td align="right">18,072</td><td align="right">96.9</td><td align="right">64</td><td align="right">18,074</td><td align="right">98.5</td></tr><tr class="evenrow team-46-9"><td>16</td><td><a href="http://www.espn.com/nba/team/_/name/gs/golden-state-warriors">Warriors</a></td><td align="right">34</td><td align="right">614,176</td><td align="right" class="sortcell">18,064</td><td align="right">100.0</td><td align="right">31</td><td align="right">17,454</td><td align="right">93.6</td><td align="right">65</td><td align="right">17,773</td><td align="right">96.9</td></tr><tr class="oddrow team-46-5"><td>17</td><td><a href="http://www.espn.com/nba/team/_/name/cle/cleveland-cavaliers">Cavaliers</a></td><td align="right">36</td><td align="right">623,576</td><td align="right" class="sortcell">17,816</td><td align="right">91.7</td><td align="right">29</td><td align="right">18,254</td><td align="right">94.9</td><td align="right">65</td><td align="right">18,014</td><td align="right">93.1</td></tr><tr class="evenrow team-46-15"><td>18</td><td><a href="http://www.espn.com/nba/team/_/name/mil/milwaukee-bucks">Bucks</a></td><td align="right">31</td><td align="right">549,036</td><td align="right" class="sortcell">17,710</td><td align="right">102.1</td><td align="right">34</td><td align="right">17,892</td><td align="right">94.9</td><td align="right">65</td><td align="right">17,805</td><td align="right">98.2</td></tr><tr class="oddrow team-46-19"><td>19</td><td><a href="http://www.espn.com/nba/team/_/name/orl/orlando-magic">Magic</a></td><td align="right">31</td><td align="right">529,870</td><td align="right" class="sortcell">17,092</td><td align="right">90.7</td><td align="right">34</td><td align="right">17,243</td><td align="right">92.2</td><td align="right">65</td><td align="right">17,171</td><td align="right">91.5</td></tr><tr class="evenrow team-46-23"><td>20</td><td><a href="http://www.espn.com/nba/team/_/name/sac/sacramento-kings">Kings</a></td><td align="right">31</td><td align="right">520,663</td><td align="right" class="sortcell">16,795</td><td align="right">95.5</td><td align="right">33</td><td align="right">17,739</td><td align="right">94.3</td><td align="right">64</td><td align="right">17,282</td><td align="right">94.9</td></tr><tr class="oddrow team-46-27"><td>21</td><td><a href="http://www.espn.com/nba/team/_/name/wsh/washington-wizards">Wizards</a></td><td align="right">32</td><td align="right">532,702</td><td align="right" class="sortcell">16,646</td><td align="right">81.8</td><td align="right">32</td><td align="right">17,587</td><td align="right">92.3</td><td align="right">64</td><td align="right">17,117</td><td align="right">86.9</td></tr><tr class="evenrow team-46-11"><td>22</td><td><a href="http://www.espn.com/nba/team/_/name/ind/indiana-pacers">Pacers</a></td><td align="right">32</td><td align="right">529,002</td><td align="right" class="sortcell">16,531</td><td align="right">92.2</td><td align="right">33</td><td align="right">17,297</td><td align="right">91.8</td><td align="right">65</td><td align="right">16,914</td><td align="right">92.0</td></tr><tr class="oddrow team-46-3"><td>23</td><td><a href="http://www.espn.com/nba/team/_/name/no/new-orleans-pelicans">Pelicans</a></td><td align="right">32</td><td align="right">528,172</td><td align="right" class="sortcell">16,505</td><td align="right">97.9</td><td align="right">32</td><td align="right">18,234</td><td align="right">96.8</td><td align="right">64</td><td align="right">17,369</td><td align="right">97.3</td></tr><tr class="evenrow team-46-17"><td>24</td><td><a href="http://www.espn.com/nba/team/_/name/bkn/brooklyn-nets">Nets</a></td><td align="right">32</td><td align="right">524,907</td><td align="right" class="sortcell">16,403</td><td align="right">92.5</td><td align="right">32</td><td align="right">17,910</td><td align="right">93.9</td><td align="right">64</td><td align="right">17,157</td><td align="right">93.2</td></tr><tr class="oddrow team-46-1"><td>25</td><td><a href="http://www.espn.com/nba/team/_/name/atl/atlanta-hawks">Hawks</a></td><td align="right">34</td><td align="right">545,453</td><td align="right" class="sortcell">16,042</td><td align="right">95.0</td><td align="right">33</td><td align="right">17,786</td><td align="right">92.9</td><td align="right">67</td><td align="right">16,901</td><td align="right">93.9</td></tr><tr class="evenrow team-46-29"><td>26</td><td><a href="http://www.espn.com/nba/team/_/name/mem/memphis-grizzlies">Grizzlies</a></td><td align="right">33</td><td align="right">523,297</td><td align="right" class="sortcell">15,857</td><td align="right">89.1</td><td align="right">32</td><td align="right">17,508</td><td align="right">93.3</td><td align="right">65</td><td align="right">16,670</td><td align="right">91.3</td></tr><tr class="oddrow team-46-21"><td>27</td><td><a href="http://www.espn.com/nba/team/_/name/phx/phoenix-suns">Suns</a></td><td align="right">35</td><td align="right">550,633</td><td align="right" class="sortcell">15,732</td><td align="right">87.1</td><td align="right">30</td><td align="right">17,660</td><td align="right">94.7</td><td align="right">65</td><td align="right">16,622</td><td align="right">90.7</td></tr><tr class="evenrow team-46-8"><td>28</td><td><a href="http://www.espn.com/nba/team/_/name/det/detroit-pistons">Pistons</a></td><td align="right">33</td><td align="right">509,463</td><td align="right" class="sortcell">15,438</td><td align="right">75.9</td><td align="right">33</td><td align="right">17,641</td><td align="right">93.7</td><td align="right">66</td><td align="right">16,539</td><td align="right">84.5</td></tr><tr class="oddrow team-46-30"><td></td><td><a href="http://www.espn.com/nba/team/_/name/cha/charlotte-hornets">Hornets</a></td><td align="right">31</td><td align="right">478,591</td><td align="right" class="sortcell">15,438</td><td align="right">80.9</td><td align="right">34</td><td align="right">17,564</td><td align="right">92.4</td><td align="right">65</td><td align="right">16,550</td><td align="right">86.9</td></tr><tr class="evenrow team-46-16"><td>30</td><td><a href="http://www.espn.com/nba/team/_/name/min/minnesota-timberwolves">Timberwolves</a></td><td align="right">32</td><td align="right">482,112</td><td align="right" class="sortcell">15,066</td><td align="right">79.4</td><td align="right">32</td><td align="right">17,463</td><td align="right">93.4</td><td align="right">64</td><td align="right">16,264</td><td align="right">86.4</td></tr><tr class="oddrow team-46-111353"><td>31</td><td>Leb</td><td align="right">0</td><td align="right">0</td><td align="right" class="sortcell">0</td><td align="right">--</td><td align="right">1</td><td align="right">0</td><td align="right">--</td><td align="right">0</td><td align="right">0</td><td align="right">--</td></tr><tr class="evenrow team-46-111386"><td></td><td>World</td><td align="right">0</td><td align="right">0</td><td align="right" class="sortcell">0</td><td align="right">--</td><td align="right">1</td><td align="right">0</td><td align="right">--</td><td align="right">0</td><td align="right">0</td><td align="right">--</td></tr><tr class="oddrow team-46-111387"><td></td><td>Usa</td><td align="right">1</td><td align="right">0</td><td align="right" class="sortcell">0</td><td align="right">--</td><td align="right">0</td><td align="right">0</td><td align="right">--</td><td align="right">0</td><td align="right">0</td><td align="right">--</td></tr><tr class="evenrow team-46-112151"><td></td><td>Gia</td><td align="right">1</td><td align="right">17,808</td><td align="right" class="sortcell">0</td><td align="right">--</td><td align="right">0</td><td align="right">0</td><td align="right">--</td><td align="right">0</td><td align="right">0</td><td align="right">--</td></tr>
					</tbody></table>'''

In [3]:
# fix HTML
soup = BeautifulSoup(avgattendance, "html.parser")
for body in soup("tbody"):
    body.unwrap()

In [4]:
avgattendance_df = pd.read_html(str(soup), flavor="bs4")
avgattendance_df = avgattendance_df[0]
avgattendance_df

,0,1,2,3,4,5,6,7,8,9,10,11
0,2020 Team,2020 Team,Home,Home,Home,Home,Road,Road,Road,Overall,Overall,Overall
1,RK,TEAM,GMS,TOTAL,AVG,PCT,GMS,AVG,PCT,GMS,AVG,PCT
2,1,76ers,31,639491,20628,102.4,34,17706,94.5,65,19122,98.5
3,2,Mavericks,34,682096,20061,104.5,33,18249,97.8,67,19169,101.2
4,3,Raptors,32,633456,19795,100.0,32,17789,94.9,64,18792,97.5
5,4,Heat,32,629771,19680,100.4,33,18127,96.8,65,18891,98.6
6,5,Trail Blazers,32,608634,19633,101.0,34,17941,96.6,66,18748,98.8
7,6,Nuggets,33,633153,19186,98.3,32,17163,92.8,65,18190,95.7
8,7,Celtics,32,610864,19089,102.5,32,18409,97.7,64,18749,100.1
9,8,Clippers,32,610176,19068,100.0,32,17855,96.2,64,18461,98.1


In [5]:
avgattendance_df = avgattendance_df.drop(avgattendance_df.index[[0,1,32,33,34,35]])

In [6]:
mapping = {avgattendance_df.columns[0]:'rank', avgattendance_df.columns[1]: 'team', avgattendance_df.columns[2]:'home_games', avgattendance_df.columns[3]: 'total_hg_attendance',avgattendance_df.columns[4]:'avg_hg_attendance', avgattendance_df.columns[5]: 'hg_pct',avgattendance_df.columns[6]:'road_games', avgattendance_df.columns[7]: 'avg_rg_attendance',avgattendance_df.columns[8]:'rg_pct', avgattendance_df.columns[9]: 'total_games', avgattendance_df.columns[10]:'avg_total_attendance', avgattendance_df.columns[11]:'combined_pct'}
avgattendance_df = avgattendance_df.rename(columns=mapping)

avgattendance_df

,rank,team,home_games,total_hg_attendance,avg_hg_attendance,hg_pct,road_games,avg_rg_attendance,rg_pct,total_games,avg_total_attendance,combined_pct
2,1,76ers,31,639491,20628,102.4,34,17706,94.5,65,19122,98.5
3,2,Mavericks,34,682096,20061,104.5,33,18249,97.8,67,19169,101.2
4,3,Raptors,32,633456,19795,100.0,32,17789,94.9,64,18792,97.5
5,4,Heat,32,629771,19680,100.4,33,18127,96.8,65,18891,98.6
6,5,Trail Blazers,32,608634,19633,101.0,34,17941,96.6,66,18748,98.8
7,6,Nuggets,33,633153,19186,98.3,32,17163,92.8,65,18190,95.7
8,7,Celtics,32,610864,19089,102.5,32,18409,97.7,64,18749,100.1
9,8,Clippers,32,610176,19068,100.0,32,17855,96.2,64,18461,98.1
10,9,Lakers,31,588907,18997,99.6,32,18877,101.9,63,18936,100.8
11,10,NY Knicks,33,620789,18811,95.0,33,17767,94.1,66,18289,94.5


In [7]:
avgattendance_df = avgattendance_df.drop(['rank'], axis=1)
avgattendance_df

,team,home_games,total_hg_attendance,avg_hg_attendance,hg_pct,road_games,avg_rg_attendance,rg_pct,total_games,avg_total_attendance,combined_pct
2,76ers,31,639491,20628,102.4,34,17706,94.5,65,19122,98.5
3,Mavericks,34,682096,20061,104.5,33,18249,97.8,67,19169,101.2
4,Raptors,32,633456,19795,100.0,32,17789,94.9,64,18792,97.5
5,Heat,32,629771,19680,100.4,33,18127,96.8,65,18891,98.6
6,Trail Blazers,32,608634,19633,101.0,34,17941,96.6,66,18748,98.8
7,Nuggets,33,633153,19186,98.3,32,17163,92.8,65,18190,95.7
8,Celtics,32,610864,19089,102.5,32,18409,97.7,64,18749,100.1
9,Clippers,32,610176,19068,100.0,32,17855,96.2,64,18461,98.1
10,Lakers,31,588907,18997,99.6,32,18877,101.9,63,18936,100.8
11,NY Knicks,33,620789,18811,95.0,33,17767,94.1,66,18289,94.5


In [8]:
# Sort by ascending student name
avgattendance_df = avgattendance_df.sort_values('team')
avgattendance_df

,team,home_games,total_hg_attendance,avg_hg_attendance,hg_pct,road_games,avg_rg_attendance,rg_pct,total_games,avg_total_attendance,combined_pct
2,76ers,31,639491,20628,102.4,34,17706,94.5,65,19122,98.5
19,Bucks,31,549036,17710,102.1,34,17892,94.9,65,17805,98.2
12,Bulls,34,639352,18804,89.9,31,17767,94.3,65,18310,91.9
18,Cavaliers,36,623576,17816,91.7,29,18254,94.9,65,18014,93.1
8,Celtics,32,610864,19089,102.5,32,18409,97.7,64,18749,100.1
9,Clippers,32,610176,19068,100.0,32,17855,96.2,64,18461,98.1
27,Grizzlies,33,523297,15857,89.1,32,17508,93.3,65,16670,91.3
26,Hawks,34,545453,16042,95.0,33,17786,92.9,67,16901,93.9
5,Heat,32,629771,19680,100.4,33,18127,96.8,65,18891,98.6
30,Hornets,31,478591,15438,80.9,34,17564,92.4,65,16550,86.9


In [9]:
avgattendance_df = avgattendance_df.reset_index(drop=True)
avgattendance_df


,team,home_games,total_hg_attendance,avg_hg_attendance,hg_pct,road_games,avg_rg_attendance,rg_pct,total_games,avg_total_attendance,combined_pct
0,76ers,31,639491,20628,102.4,34,17706,94.5,65,19122,98.5
1,Bucks,31,549036,17710,102.1,34,17892,94.9,65,17805,98.2
2,Bulls,34,639352,18804,89.9,31,17767,94.3,65,18310,91.9
3,Cavaliers,36,623576,17816,91.7,29,18254,94.9,65,18014,93.1
4,Celtics,32,610864,19089,102.5,32,18409,97.7,64,18749,100.1
5,Clippers,32,610176,19068,100.0,32,17855,96.2,64,18461,98.1
6,Grizzlies,33,523297,15857,89.1,32,17508,93.3,65,16670,91.3
7,Hawks,34,545453,16042,95.0,33,17786,92.9,67,16901,93.9
8,Heat,32,629771,19680,100.4,33,18127,96.8,65,18891,98.6
9,Hornets,31,478591,15438,80.9,34,17564,92.4,65,16550,86.9


In [10]:
avgattendance_df.columns
avgattendance_df.dtypes

team                    object
home_games              object
total_hg_attendance     object
avg_hg_attendance       object
hg_pct                  object
road_games              object
avg_rg_attendance       object
rg_pct                  object
total_games             object
avg_total_attendance    object
combined_pct            object
dtype: object

In [11]:
avgattendance_df['home_games'] = avgattendance_df['home_games'].astype(float)
avgattendance_df['hg_pct'] = avgattendance_df['hg_pct'].astype(float)
avgattendance_df['rg_pct'] = avgattendance_df['rg_pct'].astype(float)
avgattendance_df['combined_pct'] = avgattendance_df['combined_pct'].astype(float)

In [14]:
avgattendance_df['team_id'] = np.arange(len(avgattendance_df))
avgattendance_df

,team,home_games,total_hg_attendance,avg_hg_attendance,hg_pct,road_games,avg_rg_attendance,rg_pct,total_games,avg_total_attendance,combined_pct,team_id
0,76ers,31.0,639491,20628,102.4,34,17706,94.5,65,19122,98.5,0
1,Bucks,31.0,549036,17710,102.1,34,17892,94.9,65,17805,98.2,1
2,Bulls,34.0,639352,18804,89.9,31,17767,94.3,65,18310,91.9,2
3,Cavaliers,36.0,623576,17816,91.7,29,18254,94.9,65,18014,93.1,3
4,Celtics,32.0,610864,19089,102.5,32,18409,97.7,64,18749,100.1,4
5,Clippers,32.0,610176,19068,100.0,32,17855,96.2,64,18461,98.1,5
6,Grizzlies,33.0,523297,15857,89.1,32,17508,93.3,65,16670,91.3,6
7,Hawks,34.0,545453,16042,95.0,33,17786,92.9,67,16901,93.9,7
8,Heat,32.0,629771,19680,100.4,33,18127,96.8,65,18891,98.6,8
9,Hornets,31.0,478591,15438,80.9,34,17564,92.4,65,16550,86.9,9


In [15]:
connection_string = f"postgres:{password}@localhost:5432/teamvalue_db"
engine = create_engine(f'postgresql://{connection_string}')

In [16]:
# Confirm tables
engine.table_names()

['avgattendance']

In [17]:
avgattendance_df.to_sql(name='avgattendance', con=engine, if_exists='append', index=False)